In [8]:
!wget -q -nc https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb


Selecting previously unselected package cloudflared.
(Reading database ... 126441 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.9.1) ...
Setting up cloudflared (2025.9.1) ...
Processing triggers for man-db (2.10.2-1) ...


In [9]:
!streamlit run app.py --server.port 8501 & cloudflared tunnel --url http://localhost:8501


2025-09-28T01:57:31Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-09-28T01:57:31Z INF Requesting new quick Tunnel on trycloudflare.com...



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.141.205.39:8501

2025-09-28T01:57:35Z INF +--------------------------------------------------------------------------------------------+
2025-09-28T01:57:35Z INF 

In [10]:
!pip install gradio


In [13]:
import gradio as gr
import os
from PyPDF2 import PdfReader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain
from transformers import pipeline

# --- Functions ---
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

def create_faiss_vector_store(text, path="faiss_index"):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_text(text)
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.from_texts(chunks, embedding=embeddings)
    vector_store.save_local(path)

def load_faiss_vector_store(path="faiss_index"):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return FAISS.load_local(path, embeddings, allow_dangerous_deserialization=True)

def build_qa_chain(vector_store_path="faiss_index"):
    vector_store = load_faiss_vector_store(vector_store_path)
    retriever = vector_store.as_retriever()

    hf_pipeline = pipeline(
        "text2text-generation",
        model="google/flan-t5-small",   # free local model
        tokenizer="google/flan-t5-small",
        max_length=512
    )
    llm = HuggingFacePipeline(pipeline=hf_pipeline)

    qa_chain = load_qa_chain(llm, chain_type="stuff")
    return RetrievalQA(retriever=retriever, combine_documents_chain=qa_chain)

# --- Gradio App ---
qa_chain = None

def upload_pdf(file):
    global qa_chain
    text = extract_text_from_pdf(file.name)
    create_faiss_vector_store(text)
    qa_chain = build_qa_chain()
    return "✅ PDF processed. You can now ask questions."

def ask_question(question):
    global qa_chain
    if qa_chain is None:
        return "⚠️ Please upload a PDF first."
    return qa_chain.run(question)

with gr.Blocks() as demo:
    gr.Markdown("## 📚 RAG Chatbot with FAISS + Local HuggingFace Model (Free)")

    with gr.Row():
        pdf_input = gr.File(label="Upload a PDF", file_types=[".pdf"])
        upload_btn = gr.Button("Process PDF")
    upload_status = gr.Textbox(label="Status")

    with gr.Row():
        question = gr.Textbox(label="Ask a Question")
        answer = gr.Textbox(label="Answer")
        ask_btn = gr.Button("Get Answer")

    upload_btn.click(upload_pdf, inputs=pdf_input, outputs=upload_status)
    ask_btn.click(ask_question, inputs=question, outputs=answer)

demo.launch(share=True)   # share=True gives public Colab link


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cf2f85ddd6aec6125d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [14]:
import gradio as gr
import os
from PyPDF2 import PdfReader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain
from transformers import pipeline

# --- Functions ---
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

def create_faiss_vector_store(text, path="faiss_index"):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_text(text)
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.from_texts(chunks, embedding=embeddings)
    vector_store.save_local(path)

def load_faiss_vector_store(path="faiss_index"):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return FAISS.load_local(path, embeddings, allow_dangerous_deserialization=True)

def build_qa_chain(vector_store_path="faiss_index"):
    vector_store = load_faiss_vector_store(vector_store_path)
    retriever = vector_store.as_retriever()

    hf_pipeline = pipeline(
        "text2text-generation",
        model="google/flan-t5-small",   # free lightweight model
        tokenizer="google/flan-t5-small",
        max_length=512
    )
    llm = HuggingFacePipeline(pipeline=hf_pipeline)

    qa_chain = load_qa_chain(llm, chain_type="stuff")
    return RetrievalQA(retriever=retriever, combine_documents_chain=qa_chain)

# --- Gradio App ---
qa_chain = None

def upload_pdf(file):
    global qa_chain
    text = extract_text_from_pdf(file.name)
    create_faiss_vector_store(text)
    qa_chain = build_qa_chain()
    return "✅ PDF processed. You can now ask questions."

def ask_question(question):
    global qa_chain
    if qa_chain is None:
        return "⚠️ Please upload a PDF first."
    return qa_chain.run(question)

with gr.Blocks() as demo:
    gr.Markdown("## 📚 RAG Chatbot with FAISS + Local HuggingFace Model (Free)")

    with gr.Row():
        pdf_input = gr.File(label="Upload a PDF", file_types=[".pdf"])
        upload_btn = gr.Button("Process PDF")
    upload_status = gr.Textbox(label="Status")

    with gr.Row():
        question = gr.Textbox(label="Ask a Question")
        answer = gr.Textbox(label="Answer")
        ask_btn = gr.Button("Get Answer")

    upload_btn.click(upload_pdf, inputs=pdf_input, outputs=upload_status)
    ask_btn.click(ask_question, inputs=question, outputs=answer)

# 🚀 Launch in new browser tab
demo.launch(share=True, inbrowser=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f313c8caac3637da61.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
!pip install ollama
!ollama pull llama3.2


!pip install streamlit PyPDF2 langchain sentence-transformers faiss-cpu ollama



/bin/bash: line 1: ollama: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 78.1 MB/s eta 0:00:00


In [12]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:

import streamlit as st
import os
from PyPDF2 import PdfReader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import Ollama
from langchain.chains.question_answering import load_qa_chain




# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text


# Function to create FAISS vector store
def create_faiss_vector_store(text, path="faiss_index"):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_text(text)


    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.from_texts(chunks, embedding=embeddings)
    vector_store.save_local(path)


# Load FAISS vector store
def load_faiss_vector_store(path="faiss_index"):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.load_local(path, embeddings,
                 allow_dangerous_deserialization=True)
    return vector_store


# Build QA Chain
def build_qa_chain(vector_store_path="faiss_index"):
    vector_store = load_faiss_vector_store(vector_store_path)
    retriever = vector_store.as_retriever()
    # Load QA chain for combining documents
    llm = Ollama(model="llama3.2")
    qa_chain = load_qa_chain(llm, chain_type="stuff")
    qa_chain = RetrievalQA(retriever=retriever,combine_documents_chain=qa_chain)
    return qa_chain


# Streamlit App
st.title("RAG Chatbot with FAISS and LLaMA")
st.write("Upload a PDF and ask questions based on its content.")


uploaded_file = st.file_uploader("Upload your PDF file", type="pdf")


if uploaded_file is not None:
    pdf_path = f"uploaded/{uploaded_file.name}"
    os.makedirs("uploaded", exist_ok=True)


    with open(pdf_path, "wb") as f:
        f.write(uploaded_file.getbuffer())


    text = extract_text_from_pdf(pdf_path)




    st.info("Creating FAISS vector store...")
    create_faiss_vector_store(text)


    st.info("Initializing chatbot...")
    qa_chain = build_qa_chain()
    st.success("Chatbot is ready!")


if 'qa_chain' in locals():
    question = st.text_input("Ask a question about the uploaded PDF:")
    if question:
        st.info("Querying the document...")
        answer = qa_chain.run(question)
        st.success(f"Answer: {answer}")

streamlit run app.py





In [2]:
# Install dependencies
!pip install streamlit pyngrok PyPDF2 langchain faiss-cpu sentence-transformers transformers accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 103.3 MB/s eta 0:00:00


In [3]:
%%writefile app.py
import streamlit as st
import os
from PyPDF2 import PdfReader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain

from transformers import pipeline


# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    return text


# Function to create FAISS vector store
def create_faiss_vector_store(text, path="faiss_index"):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_text(text)

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.from_texts(chunks, embedding=embeddings)
    vector_store.save_local(path)


# Load FAISS vector store
def load_faiss_vector_store(path="faiss_index"):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.load_local(path, embeddings, allow_dangerous_deserialization=True)
    return vector_store


# Build QA Chain with free HuggingFace model
def build_qa_chain(vector_store_path="faiss_index"):
    vector_store = load_faiss_vector_store(vector_store_path)
    retriever = vector_store.as_retriever()

    # ✅ Use free local model (no API token required)
    hf_pipeline = pipeline(
        "text2text-generation",
        model="google/flan-t5-small",   # lightweight free model
        tokenizer="google/flan-t5-small",
        max_length=512
    )

    llm = HuggingFacePipeline(pipeline=hf_pipeline)

    qa_chain = load_qa_chain(llm, chain_type="stuff")
    qa_chain = RetrievalQA(retriever=retriever, combine_documents_chain=qa_chain)
    return qa_chain


# Streamlit App
st.title("📚 RAG Chatbot with FAISS + Local HuggingFace Model (Free)")
st.write("Upload a PDF and ask questions based on its content.")


uploaded_file = st.file_uploader("Upload your PDF file", type="pdf")

if uploaded_file is not None:
    pdf_path = f"uploaded/{uploaded_file.name}"
    os.makedirs("uploaded", exist_ok=True)

    with open(pdf_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    text = extract_text_from_pdf(pdf_path)

    st.info("Creating FAISS vector store...")
    create_faiss_vector_store(text)

    st.info("Initializing chatbot...")
    qa_chain = build_qa_chain()
    st.success("✅ Chatbot is ready!")

if 'qa_chain' in locals():
    question = st.text_input("Ask a question about the uploaded PDF:")
    if question:
        st.info("Querying the document...")
        answer = qa_chain.run(question)
        st.success(f"Answer: {answer}")


Writing app.py


In [ ]:
from pyngrok import ngrok

# Kill old tunnels
ngrok.kill()

# Open tunnel on port 8501
public_url = ngrok.connect(8501)
print("Streamlit URL:", public_url)

# Run streamlit
!streamlit run app.py --server.port 8501 &>/dev/null&



In [6]:
!npm install -g localtunnel


⠙⠹⠸⠼⠴
changed 22 packages in 678ms
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴

In [7]:
!streamlit run app.py --server.port 8501 & npx localtunnel --port 8501



⠙⠹⠸⠼⠴⠦⠧⠇

⠏⠋⠙⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.141.205.39:8501

⠦⠧⠇your url is: https://tricky-cameras-beam.loca.lt
  Stopping...
^C
